In [995]:
ROOT_DIR = '/gpfs/commons/groups/gursoy_lab/aelhussein/ot_cost/otcost_fl_rebase'
import pandas as pd
import os
import copy
import torch
import torch.nn as nn
import sys
import numpy as np
sys.path.append(f'{ROOT_DIR}/code/helper')
import pipeline as pp
import trainers as tr
import process_results as pr
import data_preprocessing as dp
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
importlib.reload(pp)
importlib.reload(dp)
importlib.reload(pr)
importlib.reload(tr)
import pickle
from multiprocessing import Pool
from torch.optim.lr_scheduler import ExponentialLR
from sklearn import metrics
from scipy.stats import bootstrap
from torchvision import models
from unet import UNet
import torch.nn.functional as F
from sklearn.utils import resample
import nibabel as nib
import torchio as tio
from torch.utils.data  import DataLoader, random_split, TensorDataset, Dataset
from torch.nn.modules.loss import _Loss

# MODEL TRAINING

In [593]:
SQUEEZE = ['Synthetic', 'Credit']
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1161]:
def get_metric(metric_name):
    metric_mapping = {
        'Synthetic': metrics.roc_auc_score,
        'Credit': metrics.f1_score,
        'Weather': metrics.r2_score}
    return metric_mapping[metric_name]

In [819]:
def loadData(DATASET, data_num, cost):
    if DATASET == 'Synthetic':
        ##load data
        X = pd.read_csv(f'{ROOT_DIR}/data/{DATASET}/data_{data_num}_{cost:.2f}.csv', sep = ' ', names = [i for i in range(13)])
        X = X.sample(200)
    elif DATASET == 'Credit':
        X = pd.read_csv(f'{ROOT_DIR}/data/{DATASET}/data_{data_num}_{cost:.2f}.csv', sep = ' ', names = [i for i in range(29)])
        X = X.sample(200)
    elif DATASET == 'Weather':
        X = pd.read_csv(f'{ROOT_DIR}/data/{DATASET}/data_{data_num}_{cost:.2f}.csv', sep = ' ', names = [i for i in range(124)])
        X = X.sample(n=500)
    ##get X and label
    y = X.iloc[:,-1]
    X = X.iloc[:,:-1]
    return X.values, y.values

In [ ]:
def check_dataloaders_equal(loader1, loader2):
    for batch1, batch2 in zip(loader1, loader2):
        data1, labels1 = batch1
        data2, labels2 = batch2
        if not (torch.equal(data1, data2) and torch.equal(labels1, labels2)):
            return False
    return True

In [1348]:
def runModel(DATASET, c, n, architecture):
    X1, y1 = loadData(DATASET, 1, c)
    X2, y2 = loadData(DATASET, 2, c)
    model, criterion, optimizer, lr_scheduler = createModel(n)
    dataloader = dp.DataPreprocessor(DATASET, BATCH_SIZE)
    if architecture == 'single':
        train_loader, val_loader, test_loader = dataloader.preprocess(X1, y1)
    elif architecture == 'joint':
        train_loader, val_loader, test_loader = dataloader.preprocess_joint(X1, y1, X2, y2)

    # Training hyperparameters
    PATIENCE = 5 
    counter = 0 
    
    # Train loop
    train_losses = []
    val_losses = []
    early_stop = False
    best_loss = float('inf')
    best_model = None
    for epoch in range(EPOCHS):
        # Training
        model.train()
        train_loss = 0
        for x, y in train_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            model.zero_grad()
            outputs = model(x)
            if DATASET in SQUEEZE:
                y = y.unsqueeze(1)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            train_loss += loss.item() 
        train_loss /= len(train_loader)
        # Validation
        model.eval() 
        val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                outputs = model(x)
                if DATASET in SQUEEZE:
                    y = y.unsqueeze(1)
                loss = criterion(outputs, y)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        # Log
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # Early stopping
        N = 2
        moving_avg_val_loss = sum(val_losses[-N:]) / N
        if val_loss < best_loss:
            best_loss = val_loss
            counter = 0 
            best_model = copy.deepcopy(model)
        else:
            if val_loss > moving_avg_val_loss:
                counter += 1
                if counter >= PATIENCE:
                    if epoch > 30:
                        early_stop = True
                        break
    #Test
    test_loss = 0
    best_model.eval()
    with torch.no_grad():
        predictions_list = []
        true_labels_list = []
        test_loss = 0
        for X, y in test_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            predictions = best_model(X)
            if DATASET in SQUEEZE:
                y = y.unsqueeze(1)
            loss = criterion(predictions, y)
            test_loss += loss.item()
            
            # Storing the predictions and true labels
            predictions_list.extend(predictions.cpu().numpy())
            true_labels_list.extend(y.cpu().numpy())

        test_loss /= len(test_loader)
        # Converting the lists to numpy arrays
        predictions_array = np.array(predictions_list)
        
        # Clipping any extreme values
        if DATASET == 'Weather':
            predictions_array = np.clip(predictions_array, -2, 2)
        # Set labels based on threshold (for f1 score)
        elif DATASET == 'Credit':
            predictions_array = (predictions_array >= 0.5).astype(int)
        true_labels_array = np.array(true_labels_list)

        # Calculating the score
        metric_assess = get_metric(DATASET)
        score = metric_assess(true_labels_array, predictions_array)
    return best_model, score, train_losses, val_losses, test_loss

In [1378]:
class ModelTrainer:
    def __init__(self, DATASET, model, criterion, optimizer, lr_scheduler, device, patience = 5):
        self.DATASET = DATASET
        self.model = model
        self.best_model = copy.deepcopy(model)
        self.criterion = criterion
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.patience = patience
        self.device = device
        self.best_loss = float('inf')
        self.counter = 0
        self.EPOCHS = 300
        self.SQUEEZE = ['Synthetic', 'Credit']
        self.pfedme = False
        self.ditto = False

    def set_loader(self, site):
        train_loader, val_loader, test_loader = site
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader

    def train_one_epoch(self, site = None):
        model, criterion, optimizer, lr_scheduler, train_loader, _, _ = self.get_objects(site)
        model.train()
        train_loss = 0
        for x, y in train_loader:
            x, y = x.to(self.device), y.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(x)
            if self.DATASET in self.SQUEEZE:
                    y = y.unsqueeze(1)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            train_loss += loss.item()
        return train_loss / len(train_loader)

    def validate(self, site = None):
        model, criterion, optimizer, _, _, val_loader, _ = self.get_objects(site)
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(self.device), y.to(self.device)
                outputs = model(x)
                if self.DATASET in self.SQUEEZE:
                    y = y.unsqueeze(1)
                loss = criterion(outputs, y)
                val_loss += loss.item()
        return val_loss / len(val_loader)

    def test(self, site = None):
        _, criterion, _, _, _, _, test_loader = self.get_objects(site)
        test_loss = 0
        self.best_model.eval()
        with torch.no_grad():
            predictions_list = []
            true_labels_list = []
            test_loss = 0
            for x, y in test_loader:
                x, y = x.to(self.device), y.to(self.device)
                predictions = self.best_model(x)
                if self.DATASET in self.SQUEEZE:
                    y = y.unsqueeze(1)
                loss = criterion(predictions, y)
                test_loss += loss.item()
                predictions_list.extend(predictions.cpu().numpy())
                true_labels_list.extend(y.cpu().numpy())
            test_loss /= len(test_loader)
            predictions_array = np.array(predictions_list)
            if self.DATASET == 'Weather':
                predictions_array = np.clip(predictions_array, -2, 2)
            elif self.DATASET == 'Credit':
                predictions_array = (predictions_array >= 0.5).astype(int)
            true_labels_array = np.array(true_labels_list)
            metric_assess = get_metric(DATASET)
            score = metric_assess(true_labels_array, predictions_array)
            return test_loss, score

    def check_early_stopping(self, val_loss, val_losses, epoch, site = None):
        model, _, _, _, _, _, test_loader = self.get_objects(site)
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model)
            self.counter = 0
        else:
            N = 2
            moving_avg_val_loss = sum(val_losses[-N:]) / N
            if val_loss > moving_avg_val_loss:
                self.counter += 1
                if self.counter >= self.patience:
                    if epoch > 30:
                        return True
        return False

    def get_metric(self):
        metric_mapping = {
            'Synthetic': metrics.roc_auc_score,
            'Credit': metrics.f1_score,
            'Weather': metrics.r2_score}
        return metric_mapping[self.DATASET]

    def get_objects(self, site_number = None):
        if site_number == None:
            return self.model, self.criterion, self.optimizer, self.lr_scheduler, self.train_loader, self.val_loader, self.test_loader
        else:
            return (self.model_1, self.criterion_1, self.optimizer_1, self.lr_scheduler_1, self.train_loader_1, self.val_loader_1, self.test_loader_1) if site_number == 1 else (self.model_2, self.criterion_2, self.optimizer_2, self.lr_scheduler_2, self.train_loader_2, self.val_loader_2, self.test_loader_2)
    
    def run(self):
        train_losses = []
        val_losses = []
        for epoch in range(self.EPOCHS):
            train_loss = self.train_one_epoch()
            train_losses.append(train_loss)
            val_loss = self.validate()
            if self.check_early_stopping(val_loss, val_losses, epoch):
                break
            val_losses.append(val_loss)
        test_loss, score = self.test()
        return score, test_loss, val_losses, train_losses


In [1379]:
class FederatedModelTrainer(ModelTrainer):
    def __init__(self, DATASET,  model, criterion, optimizer, lr_scheduler, device, patience = 5, pfedme = False, pfedme_reg =1e-1):
        super().__init__(DATASET, model,criterion, optimizer, lr_scheduler, device)
        self.model_1, self.criterion_1, self.optimizer_1, self.lr_scheduler_1 = self._clone_model()
        self.model_2, self.criterion_2, self.optimizer_2, self.lr_scheduler_2 = self._clone_model()
        total_samples_1, total_samples_2 = len(train_loader_1.dataset), len(train_loader_2.dataset)
        self.weight_1 = total_samples_1 / (total_samples_1 + total_samples_2)
        self.weight_2 = total_samples_2 / (total_samples_1 + total_samples_2)
        self.pfedme = pfedme
        self.pfedme_reg = pfedme_reg
        self.ROUNDS = 5
        self.gradient_diversity = []

    def set_loader(self, site_1_data, site_2_data):
        self.train_loader_1, self.val_loader_1, self.test_loader_1 = site_1_data
        self.train_loader_2, self.val_loader_2, self.test_loader_2 = site_1_data
    
    def _clone_model(self):
        model_clone = copy.deepcopy(self.model)
        criterion_clone = copy.deepcopy(self.criterion)
        optimizer_clone = type(self.optimizer)(model_clone.parameters(), **self.optimizer.defaults)
        lr_scheduler_clone = type(self.lr_scheduler)(optimizer_clone, gamma = self.lr_scheduler.gamma)
        return model_clone, criterion_clone, optimizer_clone, lr_scheduler_clone

    def train_site_epoch(self, site):
        model, criterion, optimizer, lr_scheduler, train_loader, _, _ = self.get_objects(site)
        model.train()
        for i in range(self.ROUNDS):
            train_loss = 0
            for x, y in train_loader:
                x, y = x.to(self.device), y.to(self.device)
                optimizer.zero_grad()
                outputs = self.model(x)
                if self.DATASET in self.SQUEEZE:
                        y = y.unsqueeze(1)
                loss = criterion(outputs, y)
                if self.pfedme:
                    loss = self.pfedme_loss(site, loss)
                loss.backward()
                optimizer.step()
                lr_scheduler.step()
                train_loss += loss.item()
        return train_loss / len(train_loader)
   
    def fed_avg(self):
        for name, param in self.model.named_parameters():
            weighted_avg_param = (self.weight_1 * self.model_1.state_dict()[name] + self.weight_2 * self.model_2.state_dict()[name]) 
            self.model.state_dict()[name].copy_(weighted_avg_param)
        self.model_1.load_state_dict(self.model.state_dict())
        self.model_2.load_state_dict(self.model.state_dict())

    def pfedme_alg(self):
        for name, param in self.model.named_parameters():
            weighted_avg_param = (self.weight_1 * self.model_1.state_dict()[name] + self.weight_2 * self.model_2.state_dict()[name]) 
            self.model.state_dict()[name].copy_(weighted_avg_param)

    def pfedme_loss(self, site, loss):
        regularization_loss = 0
        model, _, _, _, _, _, _ = self.get_objects(site)
        for p, g_p in zip(model.parameters(), self.model.parameters()):
            regularization_loss += torch.norm(p - g_p)
        regularization_loss = self.pfedme_reg * regularization_loss
        return regularization_loss

    def get_fed_learning_algorithm(self):
        if not self.pfedme:
            return self.fed_avg
        elif self.pfedme:
            return self.pfedme_alg

    def run(self):
        train_losses = []
        val_losses = []
        fed_learning = self.get_fed_learning_algorithm()
        for epoch in range(self.EPOCHS // self.ROUNDS):
            train_loss = self.train_site_epoch(1)
            train_loss +=  self.train_site_epoch(2)
            train_losses.append(train_loss)
            fed_learning()
            val_loss = self.validate(1)
            val_loss += self.validate(2)
            if self.check_early_stopping(val_loss, val_losses, epoch, 1):
                break
            val_losses.append(val_loss)
        test_loss, score = self.test(1)
        return score, test_loss, val_losses, train_losses


In [1380]:
class DittoModelTrainer(FederatedModelTrainer):
    def __init__(self, DATASET,  model, criterion, optimizer, lr_scheduler, device, patience = 5, reg = 0.1):
        super().__init__(DATASET, model,criterion, optimizer, lr_scheduler, device)
        self.model_1_s, self.criterion_1_s, self.optimizer_1_s, self.lr_scheduler_1_s = self._clone_model()
        self.model_2_s, self.criterion_2_s, self.optimizer_2_s, self.lr_scheduler_2_s = self._clone_model()
        self.ditto = True
        self.reg = reg

    def set_loader(self, site_1_data, site_2_data):
        self.train_loader_1, self.val_loader_1, self.test_loader_1 = site_1_data
        self.train_loader_2, self.val_loader_2, self.test_loader_2 = site_1_data


    def train_site_epoch(self, site):
        model, criterion, optimizer, lr_scheduler, train_loader, _, _ = self.get_sent_objects(site)
        model.train()
        for i in range(self.ROUNDS):
            train_loss = 0
            for x, y in train_loader:
                x, y = x.to(self.device), y.to(self.device)
                optimizer.zero_grad()
                outputs = self.model(x)
                if self.DATASET in self.SQUEEZE:
                        y = y.unsqueeze(1)
                loss = criterion(outputs, y)
                if self.pfedme:
                    loss = self.pfedme_loss(site, loss)
                loss.backward()
                optimizer.step()
                #lr_scheduler.step()
                train_loss += loss.item()
        return train_loss / len(train_loader)

    def train_site_personal_epoch(self, site):
        model_site_p, criterion_p, optimizer_site_p, lr_scheduler_site_p, train_loader, _ , _  = self.get_objects(site)
        model_site, criterion, optimizer_site, lr_scheduler_site, _, _, _ = self.get_sent_objects(site)
        train_loss = 0
        for i in range(self.ROUNDS):
            for x, y in train_loader:
                x, y = x.to(self.device), y.to(self.device)
                if self.DATASET in self.SQUEEZE:
                    y = y.unsqueeze(1)
                loss = self.ditto_loss(model_site, model_site_p, criterion_p, x, y)
                train_loss += loss.item()
                model_site_p.zero_grad()
                loss.backward()
                optimizer_site_p.step()
                lr_scheduler_site_p.step() 
            train_loss /= len(train_loader)
        return train_loss
    
    def ditto_loss(self, model_site, model_site_p, criterion, x, y):
        outputs = model_site_p(x.float())
        loss = criterion(outputs, y)
        regularization_loss = 0
        for p, g_p in zip(model_site.parameters(), model_site_p.parameters()):
            regularization_loss += torch.norm(p - g_p)
            regularization_loss = self.reg * regularization_loss
        loss = loss + regularization_loss
        return loss

    def get_sent_objects(self, site_number):
        return (self.model_1_s, self.criterion_1_s, self.optimizer_1_s, self.lr_scheduler_1_s,self.train_loader_1, self.val_loader_1, self.test_loader_1 ) if site_number == 1 else (self.model_2_s, self.criterion_2_s, self.optimizer_2_s, self.lr_scheduler_2_s, self.train_loader_2, self.val_loader_2, self.test_loader_2)

    def run(self):
        train_losses = []
        val_losses = []
        fed_learning = self.get_fed_learning_algorithm()
        for epoch in range(self.EPOCHS // self.ROUNDS):
            _ = self.train_site_epoch(1)
            _ =  self.train_site_epoch(2)
            train_loss = self.train_site_personal_epoch(1)
            train_loss +=  self.train_site_personal_epoch(2)
            train_losses.append(train_loss)
            fed_learning()
            val_loss = self.validate(1)
            val_loss += self.validate(2)
            if self.check_early_stopping(val_loss, val_losses, epoch, 1):
                break
            val_losses.append(val_loss)
        test_loss, score = self.test(1)
        return score, test_loss, val_losses, train_losses


In [1381]:
def modelRuns(DATASET, c, n):
    scores = {'single':[], 'joint':[], 'federated':[], 'pfedme':[], 'ditto':[]}
    test_losses = {'single':[], 'joint':[], 'federated':[], 'pfedme':[], 'ditto':[]}
    val_losses = {'single':[], 'joint':[], 'federated':[], 'pfedme':[], 'ditto':[]}
    train_losses = {'single':[], 'joint':[], 'federated':[], 'pfedme':[], 'ditto':[]}
    X1, y1 = loadData(DATASET, 1, c)
    X2, y2 = loadData(DATASET, 2, c)

    for run in range(RUNS):
        #single
        arch = 'single'
        model, criterion, optimizer, lr_scheduler = createModel(n)
        dataloader = dp.DataPreprocessor(DATASET, BATCH_SIZE)
        site_1 = dataloader.preprocess(X1, y1)
        trainer = ModelTrainer(DATASET, model, criterion, optimizer, lr_scheduler, DEVICE)
        trainer.set_loader(site_1)
        scores[arch], test_losses[arch], val_losses[arch], train_losses[arch] = trainer.run()

        #joint
        arch = 'joint'
        model, criterion, optimizer, lr_scheduler = createModel(n)
        dataloader = dp.DataPreprocessor(DATASET, BATCH_SIZE)
        site_joint = dataloader.preprocess_joint(X1, y1, X2, y2)
        trainer = ModelTrainer(DATASET, model, criterion, optimizer, lr_scheduler, DEVICE)
        trainer.set_loader(site_joint)
        scores[arch], test_losses[arch], val_losses[arch], train_losses[arch] = trainer.run()

        #federated
        arch = 'federated'
        model, criterion, optimizer, lr_scheduler = createModel(n)
        dataloader = dp.DataPreprocessor(DATASET, BATCH_SIZE)
        site_1 = dataloader.preprocess(X1, y1)
        site_2 = dataloader.preprocess(X2, y2)
        trainer = FederatedModelTrainer(DATASET, model, criterion, optimizer, lr_scheduler, DEVICE)
        trainer.set_loader(site_1, site_2)
        scores[arch], test_losses[arch], val_losses[arch], train_losses[arch] = trainer.run()

        #pfedme
        arch = 'pfedme'
        model, criterion, optimizer, lr_scheduler = createModel(n)
        dataloader = dp.DataPreprocessor(DATASET, BATCH_SIZE)
        site_1 = dataloader.preprocess(X1, y1)
        site_2 = dataloader.preprocess(X2, y2)
        trainer = FederatedModelTrainer(DATASET, model, criterion, optimizer, lr_scheduler, DEVICE, pfedme = True)
        trainer.set_loader(site_1, site_2)
        scores[arch], test_losses[arch], val_losses[arch], train_losses[arch] = trainer.run()
        
        #ditto
        arch = 'ditto'
        model, criterion, optimizer, lr_scheduler = createModel(n)
        dataloader = dp.DataPreprocessor(DATASET, BATCH_SIZE)
        site_1 = dataloader.preprocess(X1, y1)
        site_2 = dataloader.preprocess(X2, y2)
        trainer = DittoModelTrainer(DATASET, model, criterion, optimizer, lr_scheduler, DEVICE)
        trainer.set_loader(site_1, site_2)
        scores[arch], test_losses[arch], val_losses[arch], train_losses[arch] = trainer.run()

    return scores, train_losses, val_losses, test_losses

In [1337]:
def runAnalysis(DATASET, costs, n):
    results_scores = {}
    results_train_losses = {}
    results_val_losses = {}
    results_test_losses = {}
    for c in costs:
        results_scores[c], results_train_losses[c], results_val_losses[c], results_test_losses[c] =  modelRuns(DATASET, c, n)

    with open(f'{ROOT_DIR}/results/{DATASET}_scores_full.pkl', 'wb') as f:
        pickle.dump(results_scores, f)
    
    with open(f'{ROOT_DIR}/results/{DATASET}_train_losses_full.pkl', 'wb') as f:
        pickle.dump(results_train_losses, f)

    with open(f'{ROOT_DIR}/results/{DATASET}_val_losses_full.pkl', 'wb') as f:
        pickle.dump(results_val_losses, f)
    
    with open(f'{ROOT_DIR}/results/{DATASET}_test_losses_full.pkl', 'wb') as f:
        pickle.dump(results_test_losses, f)

    return results_scores, results_train_losses, results_val_losses, results_test_losses

In [1151]:
def modelRuns(DATASET, c, n, architecture):
    scores = []
    train_loss_list = []
    val_loss_list = []
    test_loss_list = []

    for run in range(RUNS):
        best_model, score, train_losses, val_losses, test_loss = runModel(DATASET, c, n, architecture)
        scores.append(score)
        train_loss_list.append(train_losses)
        val_loss_list.append(val_losses)
        test_loss_list.append(test_loss)
    return scores, train_loss_list, val_loss_list, test_loss_list

In [1150]:
def runAnalysis(DATASET, costs, n):
    results_scores = {}
    results_train_losses = {}
    results_val_losses = {}
    results_test_losses = {}
    for c in costs:
        results_scores[c] = {}
        results_train_losses[c] = {}
        results_val_losses[c] = {}
        results_test_losses[c] = {}
        for architecture in ['single', 'joint']:
            scores, train_loss_list, val_loss_list, test_loss_list = modelRuns(DATASET, c, n, architecture)
            results_scores[c][architecture] = scores
            results_train_losses[c][architecture] = train_loss_list
            results_val_losses[c][architecture] = val_loss_list
            results_test_losses[c][architecture] = test_loss_list
    '''
    with open(f'{ROOT_DIR}/results/{DATASET}_scores.pkl', 'wb') as f:
        pickle.dump(results_scores, f)
    
    with open(f'{ROOT_DIR}/results/{DATASET}_train_losses.pkl', 'wb') as f:
        pickle.dump(results_train_losses, f)

    with open(f'{ROOT_DIR}/results/{DATASET}_val_losses.pkl', 'wb') as f:
        pickle.dump(results_val_losses, f)
    
    with open(f'{ROOT_DIR}/results/{DATASET}_test_losses.pkl', 'wb') as f:
        pickle.dump(results_test_losses, f)
    '''


    return results_scores, results_train_losses, results_val_losses, results_test_losses

In [600]:
def plotter(auc_scores,train_loss_list, val_loss_list, test_loss_list, data_type):    
    # Plotting Train Losses
    plt.figure(figsize = (5,3))
    # Determine the maximum length
    train_loss_list = [t[5:] for t in train_loss_list]
    val_loss_list = [v[5:] for v in val_loss_list]
    max_length = max(max(len(t) for t in train_loss_list), max(len(v) for v in val_loss_list))
    # Pad the shorter lists with np.nan
    train_losses_padded = [np.pad(t, (0, max_length - len(t)), 'constant', constant_values=np.nan) for t in train_loss_list]
    val_losses_padded = [np.pad(v, (0, max_length - len(v)), 'constant', constant_values=np.nan) for v in val_loss_list]
    train_losses = pd.DataFrame(train_losses_padded)
    val_losses = pd.DataFrame(val_losses_padded)
    sns.lineplot(train_losses.mean(axis = 0), label = 'Train', alpha = 0.5)
    sns.lineplot(val_losses.mean(axis = 0), label = 'Val', alpha = 0.5)
    plt.title('Training and Validation Losses')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    return



## Synthetic

In [1339]:
class Feedforward(torch.nn.Module):
        def __init__(self, input_size):
                super(Feedforward, self).__init__()
                self.input_size = input_size
                self.hidden_size = [56, 6]
                self.fc = torch.nn.Sequential(nn.Linear(self.input_size, self.hidden_size[0]),
                                                nn.ReLU(),
                                                nn.Dropout(0.3),
                                                nn.Linear(self.hidden_size[0], self.hidden_size[1]),
                                                nn.ReLU(),
                                                nn.Linear(self.hidden_size[1], 1))
                self.sigmoid = torch.nn.Sigmoid()
                for layer in self.fc:
                        if isinstance(layer, nn.Linear):
                                nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
                                nn.init.constant_(layer.bias, 0)

        def forward(self, x):
                output = self.fc(x)
                output = self.sigmoid(output)
                return output
        
def createModel(n):
    model = Feedforward(n)
    model.to(DEVICE)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
    lr_scheduler = ExponentialLR(optimizer, gamma=0.8)
    return model, criterion, optimizer, lr_scheduler

In [1386]:
EPOCHS = 300
BATCH_SIZE = 2000
RUNS = 100
DATASET = 'Synthetic'
METRIC_TEST = 'AUC'
LEARNING_RATE = 5e-2
costs = [0.03, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60]
n = 12

In [1387]:
results_scores, results_train_losses, results_val_losses, results_test_losses = runAnalysis(DATASET, costs, n)

## Credit

In [ ]:
class Feedforward(torch.nn.Module):
        def __init__(self, input_size):
                super(Feedforward, self).__init__()
                self.input_size = input_size
                self.hidden_size  = [56,56,28]
                self.fc = nn.Sequential(
                        nn.Linear(self.input_size, self.hidden_size[0]),
                        nn.BatchNorm1d(self.hidden_size[0]),
                        nn.ReLU(),
                        nn.Dropout(0.5),
                        nn.Linear(self.hidden_size[0], self.hidden_size[1]),
                        nn.BatchNorm1d(self.hidden_size[1]),
                        nn.ReLU(),
                        nn.Dropout(0.5),
                        nn.Linear(self.hidden_size[1], self.hidden_size[2]),
                        nn.BatchNorm1d(self.hidden_size[2]),
                        nn.ReLU(),
                        nn.Linear(self.hidden_size[2], 1)
                )
                for layer in self.fc:
                    if isinstance(layer, nn.Linear):
                            nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
                            nn.init.constant_(layer.bias, 0)

                self.sigmoid = torch.nn.Sigmoid()

        def forward(self, x):
                output = self.fc(x)
                #output = self.sigmoid(output)
                return output
        
def createModel(n):
    model = Feedforward(28)
    model.to(DEVICE)
    criterion = nn.BCEWithLogitsLoss()
    #criterion = WeightedFocalLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
    lr_scheduler = ExponentialLR(optimizer, gamma=0.9)
    return model, criterion, optimizer, lr_scheduler


In [ ]:
EPOCHS = 300
BATCH_SIZE = 2000
RUNS = 100
DATASET = 'Credit'
METRIC_TEST = 'AUPRC'
LEARNING_RATE = 5e-2

In [ ]:
costs = [0.12, 0.23, 0.30, 0.40]
n = 28

In [ ]:
results_scores, results_train_losses, results_val_losses, results_test_losses = runAnalysis(DATASET, costs, n)

## Weather

In [ ]:
class Feedforward(torch.nn.Module):
        def __init__(self, input_size):
                super(Feedforward, self).__init__()
                self.input_size = input_size
                self.hidden_size  = [123,123,50]
                self.fc = nn.Sequential(
                        nn.Linear(self.input_size, self.hidden_size[0]),
                        nn.BatchNorm1d(self.hidden_size[0]),
                        nn.ReLU(),
                        nn.Dropout(0.5),
                        nn.Linear(self.hidden_size[0], self.hidden_size[1]),
                        nn.BatchNorm1d(self.hidden_size[1]),
                        nn.ReLU(),
                        nn.Dropout(0.5),
                        nn.Linear(self.hidden_size[1], self.hidden_size[2]),
                        nn.BatchNorm1d(self.hidden_size[2]),
                        nn.ReLU(),
                        nn.Linear(self.hidden_size[2], 1)
                )
                for layer in self.fc:
                        if isinstance(layer, nn.Linear):
                                nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
                                nn.init.constant_(layer.bias, 0)


        def forward(self, x):
                output = self.fc(x)
                return output
        
def createModel(n):
    model = Feedforward(n)
    model.to(DEVICE)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
    lr_scheduler = ExponentialLR(optimizer, gamma=0.8)
    return model, criterion, optimizer, lr_scheduler

In [ ]:
EPOCHS = 100
BATCH_SIZE = 4000
RUNS = 100
DATASET = 'Weather'
METRIC_TEST = 'R2'
LEARNING_RATE = 1e-3

In [ ]:
costs = [0.11, 0.19, 0.30, 0.40, 0.48]
c = costs[0]
n = 123

In [ ]:
results_scores, results_train_losses, results_val_losses, results_test_losses = runAnalysis(DATASET, costs, n)